In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
city_data = pd.read_csv('output_data/cities.csv')

city_data.set_index("City ID", inplace=True)

city_data.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
City ID,,,,,,,,,
0,Port Lincoln,AU,-34.7333,135.8667,65.46,80,100,15.97,1644074665
1,Lerwick,GB,60.1545,-1.1494,42.80,72,100,35.72,1644074737
2,Aklavik,CA,68.2191,-135.0107,-33.77,67,0,6.91,1644074575
3,Bambanglipuro,ID,-7.9500,110.2833,63.61,84,90,6.53,1644074738
4,Ushtobe,KZ,45.2514,77.9838,17.91,84,5,7.96,1644074738


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Set up heatmap
fig = gmaps.figure()
locations = city_data[["Latitude", "Longitude"]]
fig.add_layer(gmaps.heatmap_layer(locations))

In [19]:
#Set the Latitude and Longitude as locations and Humidity as the weight
locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity (%)"]

figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Add heatmap layer to map
fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(locations, weight, dissipating = True, max_intensity = 110, point_radius = 5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [6]:
#Narrow down data frame to ideal vacation conditions
vacay_temp_min = 75.0
vacay_temp_max = 87.5
wind_speed_max = 10.0
humidity_max = 75
cloud_max = 20.0

#Create data frame from only the ideal vacation locations
vacation_cities = city_data[(city_data["Max Temp (F)"] >= vacay_temp_min) & 
                            (city_data["Max Temp (F)"] < vacay_temp_max) &
                            (city_data["Wind Speed (mph)"] <= wind_speed_max) &
                            (city_data["Humidity (%)"] < humidity_max) &
                            (city_data["Cloudiness (%)"] < cloud_max)                           
                           ]

#Confirm that the "ideal" parameters are not too narrow
len(vacation_cities.index)

11

## Hotel Map

In [7]:
# Find closest hotel/lodging

#Create a data frame that will include hotel inforamtion
hotel_df = vacation_cities.copy()

#Add empty columns for "Hotel Name" and "Hotel Rating" to data frame
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Rating"] = ""

# Base url for gmaps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up parameter dictionary. Note: "lodging" is listed as a "Place Type" in gmaps API documentation, "hotel" is not
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

#Loop through cities in the hotel_df data frame to look for hotels
for index, row in hotel_df.iterrows():

    #Use the city's latitude and longitude to find the nearest hotel/lodging
    city_lat = row["Latitude"]
    city_lon = row["Longitude"]

    #append lat, lon to list of parameters
    params["location"] = f"{city_lat},{city_lon}"
    
    #get data from gmaps
    response = requests.get(base_url, params=params).json()  

    try:       
        #Add first hotel result for each city to data frame        
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Rating"] = response["results"][0]["rating"]
        print(f"Retrieving results for city {index}: {row[0]}")
        
    except (KeyError, IndexError):
        print(f"Missing field. Skipping {row[0]}.")

#Drop data frame rows with blanks
hotel_df = hotel_df[(hotel_df["Hotel Name"] != '') & (hotel_df["Hotel Rating"] != '')]   

#Convert ratings from objects to numbers
hotel_df["Hotel Rating"] = pd.to_numeric(hotel_df["Hotel Rating"], errors='coerce')

#Only keep the top 10 rated hotels, ensures that no more than 10 hotels are pinned on the map
hotel_df = hotel_df.nlargest(10, "Hotel Rating", keep='first')

#Display the data frame
hotel_df

Retrieving results for city 166: Mandurah
Retrieving results for city 173: Sur
Retrieving results for city 236: Barra de Tecoanapa
Retrieving results for city 245: Diego de Almagro
Retrieving results for city 304: San Miguel
Retrieving results for city 310: São Filipe
Missing field. Skipping Buin.
Retrieving results for city 357: Bubaque
Retrieving results for city 482: Vallenar
Retrieving results for city 488: San Felipe
Retrieving results for city 502: Mount Isa


,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name,Hotel Rating
City ID,,,,,,,,,,,
482,Vallenar,CL,-28.5708,-70.7581,82.06,30,0,5.26,1644074865,Humacao Bed & Breakfast,4.6
245,Diego de Almagro,CL,-26.3667,-70.0500,79.07,19,0,8.59,1644074799,hotel luna del desierto,4.3
304,San Miguel,SV,13.4833,-88.1833,82.06,69,0,2.30,1644074816,Hotel Tropico Inn San Miguel,4.2
488,San Felipe,VE,10.3399,-68.7425,82.45,48,15,7.58,1644074867,Posada Llovizna Rodríguez,4.2
236,Barra de Tecoanapa,MX,16.5167,-98.7500,76.80,55,2,1.12,1644074797,Hotel Manglares,4.0
502,Mount Isa,AU,-20.7333,139.5000,78.57,34,0,8.05,1644074569,ibis Styles Mt Isa Verona,3.9
173,Sur,OM,22.5667,59.5289,75.83,45,2,1.97,1644074780,Sur Plaza Hotel,3.8
357,Bubaque,GW,11.2833,-15.8333,79.27,45,0,7.81,1644074831,"Casa Dora, Bubaque",3.8
166,Mandurah,AU,-32.5500,115.7000,78.22,72,0,7.56,1644074778,Atrium Hotel Mandurah,3.5
